In [1]:
import numpy as np
import pygsti as pig
import matplotlib.pyplot as plt
import random
import time
import pandas
from scipy.linalg import expm, sqrtm
from tqdm import tqdm
# from qutip import *

from pygsti.modelmembers.operations import LindbladErrorgen as _LinbladErrorgen
from pygsti.modelmembers.operations import EmbeddedErrorgen as _EmbeddedErrorgen
from pygsti.modelmembers.operations import ExpErrorgenOp as _ExpErrorgenOp
from pygsti.modelmembers.operations import ComposedOp as _ComposedOp
import  pygsti.modelmembers as _mdmb
import pygsti.processors as _proc

from pygsti.tools import pdftools as _pdftools

from pygsti.circuits.gstcircuits import create_lsgst_circuits

from pygsti.models.memberdict import OrderedMemberDict as _OrderedMemberDict
from pygsti.baseobjs.label import Label as _Label, CircuitLabel as _CircuitLabel
from pygsti.circuits import Circuit
import pygsti.circuits as _circ
import pygsti.models as _mdl
from pygsti.modelmembers import operations as _op

In [2]:
def random_circuit(depth, proc_spec):
    """
    works for any 1-qubit model pack and the XYZICnot 2-qubit pack
    """
    layers = []
    for i in range(depth):
        gate = random.choice(proc_spec.gate_names)
        if gate == '(idle)':
            pass
        elif gate == 'Gcnot':
            layers.append((gate, 0, 1))
        else:
            layers.append((gate, random.choice(proc_spec.qubit_labels)))
    return Circuit(layers, proc_spec.qubit_labels)

In [3]:
def vector_from_outcomes(outcomes, num_outcomes):
    vecout = np.zeros((num_outcomes))
    for key in outcomes.keys():
        vecout[int(key[0], 2)] = outcomes[key]
    return(vecout)

def matrix_from_jacob(jacob, num_outcomes):
    matout = np.zeros((num_outcomes, len(jacob['0'*int(np.log2(num_outcomes))])))
    for key in jacob.keys():
        matout[int(key[0], 2), :] = np.array(jacob[key])
    return matout

def tensor_from_hessian(hessian, hilbert_dims):
    """
    returns a 3d-array that when dotted into the state returns the jacobian 
    """
    num_params = len(hessian['0'*int(np.log2(hilbert_dims))])
    tensor_out = np.zeros((hilbert_dims, num_params, num_params))
    for key in hessian.keys():
        tensor_out[int(key[0], 2), :, :] = hessian[key]
    return tensor_out

In [4]:
def compare_models(true_model, model1, model2, circuit_list, name1='x', name2='y'):
    distribution1 = []
    distribution2 = []
    mx = 0
    for idx, circ in enumerate(circuit_list): 
        hilbert_dims = 2**circ.width
        true_outcomes = true_model.probabilities(circ)
        outcome_set1 = model1.probabilities(circ)
        outcome_set2 = model2.probabilities(circ)
        dist1 = _pdftools.tvd(true_outcomes, outcome_set1)
        dist2 = _pdftools.tvd(true_outcomes, outcome_set2)
        if dist1 > mx:
            mx = dist1
        if dist2 > mx: 
            mx = dist2
        vtrue = vector_from_outcomes(true_outcomes, hilbert_dims)
        v1 = vector_from_outcomes(outcome_set1, hilbert_dims)
        v2 = vector_from_outcomes(outcome_set2, hilbert_dims)
        distribution1.append(vtrue - v1)
        distribution2.append(vtrue - v2)
        plt.scatter(dist1, dist2)
    plt.xlabel(name1)
    plt.ylabel(name2)
    plt.plot((0, mx), (0, mx), c='black')
    plt.show()
    return (distribution1, distribution2)

In [5]:
def calculate_jacobs(circ_list, model):
    jacobs = {}
    for circ in circ_list:
        hilbert_dims = 2**circ.width
        jacobs[circ] = matrix_from_jacob(model.sim.dprobs(circ), hilbert_dims)
    return jacobs

In [6]:
from pygsti.processors import CliffordCompilationRules as CCR

def make_rb_param(noise_model, model_pack, length_powers=5, circuits_at_length=10):
    pspec = model_pack.processor_spec()
    
    depths = [2**i for i in range(length_powers)]
    
    compilations = {'absolute': CCR.create_standard(pspec, 'absolute', ('paulis', '1Qcliffords'), verbosity=0),            
                'paulieq': CCR.create_standard(pspec, 'paulieq', ('1Qcliffords', 'allcnots'), verbosity=0)}

    design = pig.protocols.DirectRBDesign(model_pack.processor_spec(), compilations, depths, circuits_at_length, qubit_labels=model_pack.processor_spec().qubit_labels, sampler='edgegrab', 
                                           samplerargs=[0.5], randomizeout=True,
                                           citerations=20)
    pig.io.write_empty_protocol_data(design, 'RB_Data', clobber_ok=True)
    pig.io.fill_in_empty_dataset_with_fake_data(noise_model, 'RB_Data/data/dataset.txt', num_samples=1000)
    data = pig.io.load_data_from_dir('RB_Data')
    protocol = pig.protocols.RB()
    results = protocol.run(data)
    return (results.fits['full'].estimates['r'], data)

In [7]:
def make_dirichlet_covar(counts, hilbert_dims):
    total_counts = sum([counts[key] for key in counts.keys()])    
    count_vec = np.ones(hilbert_dims)
    for key in counts.keys():
        count_vec[int(key[0], 2)] += counts[key]
    prefactor = 1/( (total_counts + hilbert_dims)**2 * (total_counts + hilbert_dims + 1) )
    meas_covar = prefactor*(
        (total_counts + hilbert_dims)*np.diag(count_vec) - np.outer(count_vec, count_vec)
    )
    return meas_covar

In [8]:
def make_multinom_covar(prob_vec):
    return np.eye(len(prob_vec)) - np.outer(prob_vec, prob_vec)

In [9]:
from scipy.linalg import block_diag

class ExtendedKalmanFilter():
    def __init__(self, model, P0):
        self.model = model
        self.P = P0
        
    def update(self, circ_list, data_set, jdict, hdict, stab_noise=None, max_itr=1, itr_eps=1e-4):
        
        
        prior_covar = self.P
        prior_state = self.model.to_vector()
        
        for itr in range(max_itr):
            Smat = np.zeros((0,0))
            total_innov = np.zeros((0))
            total_jac = np.zeros((0,len(self.model.to_vector())))
        
            for circ in circ_list:
                counts = data_set[circ].counts
                total_counts = sum([counts[key] for key in counts.keys()])    
                hilbert_dims = 2**(circ.width)
                prior_state = self.model.to_vector()

                # 0) find the mean estimate for the circuit outcome under the self.model
                p_model = np.ones(hilbert_dims)
                probs = self.model.probabilities(circ)
                for key in probs.keys():
                    p_model[int(key[0], 2)] += total_counts*probs[key]
                p_model = (1/(total_counts+hilbert_dims))*p_model

                # calculate jacobian
                jacob = jdict[circ] + hdict[circ]@prior_state

                # 1) calculate your observation frequencies
                observation = np.ones(hilbert_dims)
                for key in counts.keys():
                    observation[int(key[0], 2)] += counts[key]
                observation = (1/(total_counts+hilbert_dims))*observation

                # 2) calculate the covaraiance of the observation and add model noise
                meas_covar = (1/(total_counts+hilbert_dims))*make_multinom_covar(observation)
                if stab_noise is not None:
                    meas_covar += stab_noise
                    
                # 3) Kalman gain
                smat = np.linalg.pinv(jacob@prior_covar@jacob.T + meas_covar, 1e-6)
                
                innovation = observation - p_model

                Smat = block_diag(Smat, smat)
                total_innov = np.hstack([total_innov, innovation])
                total_jac = np.vstack([total_jac, jacob])
            Kgain = prior_covar@total_jac.T@Smat
            post_state = prior_state + Kgain@total_innov
            self.P = prior_covar - Kgain@total_jac@prior_covar
            if np.linalg.norm(post_state - prior_state) < itr_eps:
                break
        self.model.from_vector(post_state)

In [10]:
from scipy.linalg import block_diag

class ExtendedKalmanFilter2():
    def __init__(self, model, P0):
        self.model = model
        self.P = P0
        
    def update(self, circ_list, data_set, jdict, hdict, stab_noise=None, max_itr=1, itr_eps=1e-4):
        
        
        prior_covar = self.P
        prior_state = self.model.to_vector()
        
        for itr in range(max_itr):
            Smat = np.zeros((0,0))
            total_innov = np.zeros((0))
            total_jac = np.zeros((0,len(self.model.to_vector())))
        
            for circ in circ_list:
                counts = data_set[circ].counts
                total_counts = sum([counts[key] for key in counts.keys()])    
                hilbert_dims = 2**(circ.width)
                prior_state = self.model.to_vector()

                # 0) find the mean estimate for the circuit outcome under the self.model
                p_model = np.zeros(hilbert_dims)
                probs = self.model.probabilities(circ)
                for key in probs.keys():
                    p_model[int(key[0], 2)] += probs[key]

                # calculate jacobian
                jacob = jdict[circ] + hdict[circ]@prior_state

                # 1) calculate your observation frequencies
                observation = np.ones(hilbert_dims)
                for key in counts.keys():
                    observation[int(key[0], 2)] += counts[key]
                observation = (1/(total_counts+hilbert_dims))*observation

                # 2) calculate the covaraiance of the observation and add model noise
                meas_covar = (1/(total_counts+hilbert_dims))*make_multinom_covar(observation)
                if stab_noise is not None:
                    meas_covar += stab_noise
                    
                # 3) Kalman gain
                smat = np.linalg.pinv(jacob@prior_covar@jacob.T + meas_covar, 1e-6)
                
                innovation = observation - p_model

                Smat = block_diag(Smat, smat)
                total_innov = np.hstack([total_innov, innovation])
                total_jac = np.vstack([total_jac, jacob])
            Kgain = prior_covar@total_jac.T@Smat
            post_state = prior_state + Kgain@total_innov
            self.P = prior_covar - Kgain@total_jac@prior_covar
            if np.linalg.norm(post_state - prior_state) < itr_eps:
                break
        self.model.from_vector(post_state)

In [10]:
from filterpy.kalman import KalmanFilter

class LinearKalmanFilter(KalmanFilter):
    def __init__(self, x0, P0, hilbert_dims):
        super().__init__(len(x0), hilbert_dims)
        self.x = x0
        self.P = P0
        
        self.F = np.eye(len(x0))
        self.Q = np.zeros((len(x0), len(x0)))
        
    def update_filter(self, circ, count_vec, pvec_model, jacob, stab_noise=None):
        
        total_counts = sum(count_vec)
        
        hilbert_dims = 2**circ.width
        
        prediction = (total_counts*pvec_model + np.ones(hilbert_dims))/(total_counts + hilbert_dims)
        observation = (count_vec + np.ones(hilbert_dims))/(total_counts + hilbert_dims)
        
        self.H = jacob
        shot_noise = (1/total_counts)*make_multinom_covar(observation)
        if stab_noise is not None:
            shot_noise += stab_noise
        self.R = shot_noise
        
        self.update(observation - prediction)

In [11]:
def regression_fit(circ_list, data_set, ref_model, jdict):
    pvec = np.zeros((0))
    jmat = np.zeros((0, num_params))
    
    for circ in circ_list:
        hilbert_dims = 2**circ.width
        
        counts = vector_from_outcomes(data_set[circ].counts, hilbert_dims)
        total_counts = sum(counts)
        observation = (counts + np.ones((hilbert_dims)))/(total_counts + hilbert_dims)
        
        p_model = np.ones((hilbert_dims))
        probs = ref_model.probabilities(circ)
        for key in probs.keys():
            p_model[int(key[0], 2)] += total_counts*probs[key]
        p_model = p_model/(total_counts + hilbert_dims)
        
        pvec = np.hstack((pvec, observation-p_model))
        jmat = np.vstack((jmat, jdict[circ]))
    return np.linalg.pinv(jmat, 1e-6)@pvec
        

# [1] Pick a Model

In [12]:
from pygsti.modelpacks import smq1Q_XYI as _smq1Q_XYI
from pygsti.modelpacks import smq1Q_XYZI as _smq1Q_XYZI
from pygsti.modelpacks import smq1Q_XYI as _smq1Q_XZ
from pygsti.modelpacks import smq2Q_XYZICNOT as _smq2Q_XYZICNOT
MODEL_PACK = _smq1Q_XYZI
REF_MODEL = MODEL_PACK.target_model('H+S')

# [2] Make Random Circuits and Calculate the Jacobians and Hessians

In [13]:
# make random circuits and design matrices
N_circs = 4000
depth = 64

random_circuits = []
for n in range(N_circs):
    random_circuits.append(random_circuit(random.choice(range(depth)), MODEL_PACK.processor_spec()))

jdict_random = dict()
hdict_random = dict()
for idx, circ in tqdm(enumerate(random_circuits)):
    jdict_random[circ] = matrix_from_jacob(REF_MODEL.sim.dprobs(circ), 2**circ.width)
    hdict_random[circ] = tensor_from_hessian(REF_MODEL.sim.hprobs(circ), 2**circ.width)

4000it [1:04:54,  1.03it/s]


# [2] Design Experiments (total of 20 trials) and make models

In [14]:
import random

num_trials = 20
len_trial = 1000

trial_circ_lists = []
for i in range(num_trials):
    trial_circ_lists.append(random.sample(random_circuits, len_trial))

In [15]:
linear_model = REF_MODEL.copy()
extended_model = REF_MODEL.copy()
regression_model = REF_MODEL.copy()
noise_model = MODEL_PACK.target_model()
noise_model = noise_model.depolarize(max_op_noise=0.01, max_spam_noise=0.001)
noise_model = noise_model.rotate(max_rotate=0.01)

In [16]:
def compare_model_list(noise_model, model_list, circ_list):
    avg_tvds = np.zeros((len(model_list)))
    for circ in circ_list:
        for i, model in enumerate(model_list):
            avg_tvds[i] += _pdftools.tvd(model.probabilities(circ), noise_model.probabilities(circ))
    return (1/len(circ_list))*avg_tvds

In [17]:
N_circs = 100
depth = 128

test_list = []
for n in range(N_circs):
    test_list.append(random_circuit(random.choice(range(depth)), MODEL_PACK.processor_spec()))

# [3] Run experiments

An experiment consists of 

1) make RB param and sample the circs a set number of times
2) run a set number of kalman iterations on the outcomes
3) after a set number, do a linear fit, sample 250 more random circuits, and record the average errors vs fit
4) Iterate, recording the prior innovation vs circuit index


The final plot will look like:

- A main plot with the learning curve as a function of time
- A secondary insert plot with the average error points and error bars
- a plot with the sampling error ceiling


In [18]:
num_params = len(REF_MODEL.to_vector())
hilbert_dims = 2**random_circuits[0].width

In [19]:
num_samples = 1000
resample_window = 250

max_iterations = 5
itr_epsilon = 1e-4


experimental_outcomes = dict()

innovation_lists = []
test_lists = []

ref_prediction_errors = []
linear_prediction_errors = []
extended_prediction_errors = []

test_sets = [] # list of lists of arrays of avg tvds

shot_noise_ceil = 1/np.sqrt(num_samples)



for list_index, circ_list in enumerate(trial_circ_lists):
    rb_param, data = make_rb_param(noise_model, MODEL_PACK);
    dataset = pig.data.simulate_data(noise_model, circ_list, num_samples=num_samples)

    P = 2*np.sqrt(rb_param)*np.eye(num_params)
    extended_model.from_vector(np.zeros(num_params))
    lkf = LinearKalmanFilter(np.zeros(num_params), P, hilbert_dims)
    ekf = ExtendedKalmanFilter(extended_model, P)

    ref_pred_error = np.zeros((len_trial))
    linear_pred_error = np.zeros((len_trial))
    extended_pred_error = np.zeros((len_trial))

    trial_test_list = []

    for idx, circ in tqdm(enumerate(circ_list)):
        if idx % resample_window == 0: 
            #regression_model.from_vector(regression_fit(circ_list[0:idx], dataset, REF_MODEL, jdict_random))
            trial_test_list.append(compare_model_list(noise_model, [REF_MODEL, linear_model, extended_model, regression_model], test_list))

        linear_model.from_vector(lkf.x)
        extended_model = ekf.model
        ref_pred_error[idx] = _pdftools.tvd(REF_MODEL.probabilities(circ), dataset[circ].fractions)
        linear_pred_error[idx] = _pdftools.tvd(linear_model.probabilities(circ), dataset[circ].fractions)
        extended_pred_error[idx] = _pdftools.tvd(extended_model.probabilities(circ), dataset[circ].fractions)

        count_vec = vector_from_outcomes(dataset[circ].counts, hilbert_dims)
        ref_prob = vector_from_outcomes(REF_MODEL.probabilities(circ), hilbert_dims)

        lkf.update_filter(circ, count_vec, ref_prob, jdict_random[circ], 1e-4*np.eye(hilbert_dims))
        ekf.update([circ], dataset, jdict_random, hdict_random, 1e-1*np.eye(hilbert_dims), max_iterations, itr_epsilon)

    test_sets.append(trial_test_list)
    ref_prediction_errors.append(ref_pred_error)
    linear_prediction_errors.append(linear_pred_error)
    extended_prediction_errors.append(extended_pred_error)

- Sampling 10 circuits at DRB length 1 (1 of 5 depths) with seed 842003
- Sampling 10 circuits at DRB length 2 (2 of 5 depths) with seed 842013
- Sampling 10 circuits at DRB length 4 (3 of 5 depths) with seed 842023
- Sampling 10 circuits at DRB length 8 (4 of 5 depths) with seed 842033
- Sampling 10 circuits at DRB length 16 (5 of 5 depths) with seed 842043


/home/jpmarceaux/Applications/pyGSTi/pygsti/io/writers.py:502: UserWarning: write_empty_protocol_data has recently changed its signature - it looks like you need to swap the first two arguments.  Continuing using the old signature...
  _warnings.warn(("write_empty_protocol_data has recently changed its signature - it looks like you need to swap"
/home/jpmarceaux/Applications/pyGSTi/pygsti/io/writers.py:623: UserWarning: fill_in_empty_dataset_with_fake_data has recently changed its signature - it looks like you need to swap the first two arguments.  Continuing using the old signature...
  _warnings.warn(("fill_in_empty_dataset_with_fake_data has recently changed its signature - it looks like"
/home/jpmarceaux/Applications/pyGSTi/pygsti/layouts/evaltree.py:183: UserWarning: Created an evaluation tree that is inefficient: tree-size > 3 * #circuits !
This is likely due to the fact that the circuits being simulated do not have a
periodic structure. Consider using a different forward simulat

- Sampling 10 circuits at DRB length 1 (1 of 5 depths) with seed 938239
- Sampling 10 circuits at DRB length 2 (2 of 5 depths) with seed 938249
- Sampling 10 circuits at DRB length 4 (3 of 5 depths) with seed 938259
- Sampling 10 circuits at DRB length 8 (4 of 5 depths) with seed 938269
- Sampling 10 circuits at DRB length 16 (5 of 5 depths) with seed 938279


1000it [01:51,  8.97it/s]


- Sampling 10 circuits at DRB length 1 (1 of 5 depths) with seed 845587
- Sampling 10 circuits at DRB length 2 (2 of 5 depths) with seed 845597
- Sampling 10 circuits at DRB length 4 (3 of 5 depths) with seed 845607
- Sampling 10 circuits at DRB length 8 (4 of 5 depths) with seed 845617
- Sampling 10 circuits at DRB length 16 (5 of 5 depths) with seed 845627


...

1000it [01:46,  9.41it/s]


- Sampling 10 circuits at DRB length 1 (1 of 5 depths) with seed 560589
- Sampling 10 circuits at DRB length 2 (2 of 5 depths) with seed 560599
- Sampling 10 circuits at DRB length 4 (3 of 5 depths) with seed 560609
- Sampling 10 circuits at DRB length 8 (4 of 5 depths) with seed 560619
- Sampling 10 circuits at DRB length 16 (5 of 5 depths) with seed 560629


1000it [01:49,  9.10it/s]


- Sampling 10 circuits at DRB length 1 (1 of 5 depths) with seed 756071
- Sampling 10 circuits at DRB length 2 (2 of 5 depths) with seed 756081
- Sampling 10 circuits at DRB length 4 (3 of 5 depths) with seed 756091
- Sampling 10 circuits at DRB length 8 (4 of 5 depths) with seed 756101
- Sampling 10 circuits at DRB length 16 (5 of 5 depths) with seed 756111


1000it [01:46,  9.40it/s]


In [20]:
print(rb_param)

0.014533939770035642


In [21]:
avg_ref_error = sum([evec for evec in ref_prediction_errors])/len(ref_prediction_errors)
avg_linear_error = sum([evec for evec in linear_prediction_errors])/len(linear_prediction_errors)
avg_extended_error = sum([evec for evec in extended_prediction_errors])/len(extended_prediction_errors)

In [22]:
print(len(test_sets))

20


In [23]:
print(test_sets)

[[array([0.04135732, 0.04135732, 0.04135732, 0.04135732]), array([0.04135732, 0.0297464 , 0.00387638, 0.04135732]), array([0.04135732, 0.02579353, 0.00327656, 0.04135732]), array([0.04135732, 0.01952078, 0.00281196, 0.04135732])], [array([0.04135732, 0.01064554, 0.04135732, 0.04135732]), array([0.04135732, 0.03065456, 0.00456734, 0.04135732]), array([0.04135732, 0.02695879, 0.00449693, 0.04135732]), array([0.04135732, 0.02188455, 0.00326647, 0.04135732])], [array([0.04135732, 0.01628538, 0.04135732, 0.04135732]), array([0.04135732, 0.02801617, 0.00407795, 0.04135732]), array([0.04135732, 0.01989475, 0.00284841, 0.04135732]), array([0.04135732, 0.00490174, 0.00235503, 0.04135732])], [array([0.04135732, 0.01242236, 0.04135732, 0.04135732]), array([0.04135732, 0.03066866, 0.0045896 , 0.04135732]), array([0.04135732, 0.02816704, 0.00297001, 0.04135732]), array([0.04135732, 0.0251713 , 0.00195836, 0.04135732])], [array([0.04135732, 0.02163772, 0.04135732, 0.04135732]), array([0.04135732, 0.

In [24]:
num_batches = int(np.floor(len_trial/resample_window))
avg_test_errors = [np.zeros(4)]*num_batches
mats = []
for trial in test_sets:
    mats.append(np.vstack([batch for batch in trial]))

In [25]:
avg = np.zeros(mats[0].shape)
for mat in mats:
    avg += mat/len(mats)
sigma = np.zeros(mats[0].shape)
for mat in mats:
    sigma += (mat - avg)**2/len(mats)

In [26]:
print(avg)

[[0.04135732 0.0239989  0.04135732 0.04135732]
 [0.04135732 0.03104821 0.00429133 0.04135732]
 [0.04135732 0.02872931 0.00315044 0.04135732]
 [0.04135732 0.02553258 0.00261509 0.04135732]]


In [27]:
print(sigma)

[[0.00000000e+00 6.60020012e-05 1.92592994e-34 0.00000000e+00]
 [0.00000000e+00 1.02993297e-06 5.32576091e-07 0.00000000e+00]
 [0.00000000e+00 8.28161489e-06 4.26610223e-07 0.00000000e+00]
 [0.00000000e+00 3.97223006e-05 3.42838246e-07 0.00000000e+00]]


In [37]:
%matplotlib

from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition,
                                                  mark_inset)

plt.rc('font', size=24)

fig, ax1 = plt.subplots()
plt.title(f'Average prior prediction errors on training set')
plt.ylabel('Prediction error [TVD]')
plt.xlabel('Circuit Index')


for trial in range(num_trials):
    I = [i for i in range(len_trial)]
    ax1.plot(I, avg_ref_error, c='red')
    ax1.plot(I, avg_linear_error, c='blue')
    ax1.plot(I, avg_extended_error, c='green')
    
#     for idx, val in enumerate(ref_prediction_errors[trial]):
#         ax1.scatter(idx, val, c='red', alpha=0.1)
#         ax1.scatter(idx, linear_prediction_errors[trial][idx], c='blue', alpha=0.1)
#         ax1.scatter(idx, extended_prediction_errors[trial][idx], c='green', alpha=0.1)
        

    
ax2 = plt.axes([0,0,num_trials,0.04])
ip = InsetPosition(ax1, [0.5,0.5,0.4,0.4])
ax2.set_axes_locator(ip)
ax2.set_ylim([0, 0.06])

plt.title('Average error on test set')


ax2.scatter(0, avg[0,0], c='red', marker='x')
for b in range(num_batches):
    bp = b+1
    ideal_points = ax2.scatter(resample_window*bp, avg[b, 0], c='red', marker='x')
    linear_points = ax2.scatter(resample_window*bp, avg[b, 1], c='blue', marker='v', label='Linear Kalman')
    extended_points = ax2.scatter(resample_window*bp, avg[b, 2], c='green', marker='o', label='Extended Kalman')
    
plt.rc('font', size=14)

ax2.legend([ideal_points, linear_points, extended_points], ['Ideal Model', 'Linear KF', 'Extended KF'])
#     ax2.scatter(resample_window*bp, avg_batch_errors[b][3], c='yellow')
#     ax2.errorbar(resample_window*bp, avg_batch_errors[b][3], batch_sigma[b][3], c='yellow')




Using matplotlib backend: Qt5Agg


In [29]:
print(rb_param)

0.03537786258200021


In [67]:
f

NameError: name 'f' is not defined

In [26]:
f

NameError: name 'f' is not defined